## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

In [ ]:
import pandas as pd
import numpy as np
import io

from sklearn.metrics import * 
from sklearn.model_selection import train_test_split 

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving sms_spam.csv to sms_spam.csv


In [ ]:
spam = pd.read_csv(io.BytesIO(uploaded['sms_spam.csv']))
spam

,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or £10,000 ..."
4,spam,okmail: Dear Dave this is your final notice to...
...,...,...
5554,ham,You are a great role model. You are giving so ...
5555,ham,"Awesome, I remember the last time we got someb..."
5556,spam,"If you don't, your prize will go to another cu..."
5557,spam,"SMS. ac JSco: Energy is high, but u may not kn..."


Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

**Решение**

1) токенизация

In [ ]:
import nltk # уже знакомая нам библиотека nltk
from nltk.tokenize import word_tokenize # готовый токенизатор библиотеки nltk

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
word_tokenized_list = []
for sms in spam.text:
  words = word_tokenize(sms)
  new_words = [word for word in words if word.isalnum()]
  word_tokenized_list.append(new_words)
len(word_tokenized_list)

5559

2) приведение к нижнему регистру

In [ ]:
for i in range(len(word_tokenized_list)):
  for j in range(len(word_tokenized_list[i])):
    word_tokenized_list[i][j] = word_tokenized_list[i][j].lower()

3) удаление стоп-слов

In [ ]:
# импортируем стоп-слова из библиотеки nltk
from nltk.corpus import stopwords
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
noise = stopwords.words("english")

word_list_without_noize = []
for i in range(len(word_tokenized_list)):
  element = []
  for j in range(len(word_tokenized_list[i])):
    if word_tokenized_list[i][j] not in noise:
      element.append(word_tokenized_list[i][j])
  word_list_without_noize.append(element)

4) лемматизация

In [ ]:
# устанавливаем pymorphy2
!pip install pymorphy2

     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 8.2 MB 48.7 MB/s 


In [ ]:
from pymorphy2 import MorphAnalyzer
pymorphy2_analyzer = MorphAnalyzer()

In [ ]:
word_list_with_lemmatization = []
for sentence in word_list_without_noize:
  current_sentence = ""
  for word in sentence:
    current_sentence = current_sentence + " " + pymorphy2_analyzer.parse(word)[0].normal_form
  word_list_with_lemmatization.append(current_sentence)


5) векторазирация

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(word_list_with_lemmatization, spam.type, train_size = 0.7)

In [ ]:
from sklearn.naive_bayes import MultinomialNB # наивный байесовский классификатор
from sklearn.feature_extraction.text import CountVectorizer # модель "мешка слов", см. далее

In [ ]:
history = []

In [ ]:
median_length_word = 0
words = 0
for sentence in word_list_with_lemmatization:
  for word in sentence:
    median_length_word += len(word)
  words += len(sentence.split())
median_length_word /= words
median_length_word

5.692830312609043

5.1) мешок n-грамм

In [ ]:
for i in range (2, 6):
  for j in range (i, 6):
    vectorizer = CountVectorizer(ngram_range=(i, j))
    vectorized_x_train = vectorizer.fit_transform(x_train)
    clf = MultinomialNB()
    clf.fit(vectorized_x_train, y_train)

    vectorized_x_test = vectorizer.transform(x_test)
    pred = clf.predict(vectorized_x_test)
    print(f"n-framm from {i} to {j}")
    print(classification_report(y_test, pred))

    history_element = []
    history_element.append("CountVectorizer")
    history_element.append(1.0)
    history_element.append(1.0)
    history_element.append(None)
    history_element.append(f"({i}, {j})")
    weighted_avg = precision_recall_fscore_support(y_test, pred, average='weighted')
    history_element.append(weighted_avg[0])
    history_element.append(weighted_avg[1])
    history_element.append(weighted_avg[2])
    history_element.append(accuracy_score(y_test, pred))
    history.append(history_element)

n-framm from 2 to 2
              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1428
        spam       0.97      0.78      0.87       240

    accuracy                           0.97      1668
   macro avg       0.97      0.89      0.92      1668
weighted avg       0.97      0.97      0.96      1668

n-framm from 2 to 3
              precision    recall  f1-score   support

         ham       0.97      1.00      0.98      1428
        spam       0.97      0.79      0.87       240

    accuracy                           0.97      1668
   macro avg       0.97      0.89      0.92      1668
weighted avg       0.97      0.97      0.96      1668

n-framm from 2 to 4
              precision    recall  f1-score   support

         ham       0.97      1.00      0.98      1428
        spam       0.97      0.79      0.87       240

    accuracy                           0.97      1668
   macro avg       0.97      0.89      0.92      1668
weighted avg  

5.2) tf-idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
for i in range (1, 4):
  for j in range (i, 4):
    for max_features in [500, 1500, 4000]:
      for min_df in [0.0, 0.0025]:
        for max_df in [0.01, 1.0]:
          if (max_df < min_df):
            continue
          
          tfidf_vectorizer = TfidfVectorizer(ngram_range=(i, j), max_df=max_df, min_df=min_df, max_features=max_features)
          tfidf_vectorized_x_train = tfidf_vectorizer.fit_transform(x_train)

          clf = MultinomialNB()
          clf.fit(tfidf_vectorized_x_train, y_train)

          tfidf_vectorized_x_test = tfidf_vectorizer.transform(x_test)

          # получаем предсказания и выводим информацию о качестве
          pred = clf.predict(tfidf_vectorized_x_test)
          print(f"n-framm from {i} to {j}")
          print(f"min_df = {min_df}; max_df = {max_df}; max_features = {max_features}")
          print(classification_report(y_test, pred))

          history_element = []
          history_element.append("TfidfVectorizer")
          history_element.append(min_df)
          history_element.append(max_df)
          history_element.append(max_features)
          history_element.append(f"({i}, {j})")
          weighted_avg = precision_recall_fscore_support(y_test, pred, average='weighted')
          history_element.append(weighted_avg[0])
          history_element.append(weighted_avg[1])
          history_element.append(weighted_avg[2])
          history_element.append(accuracy_score(y_test, pred))
          history.append(history_element)

n-framm from 1 to 1
min_df = 0.0; max_df = 0.01; max_features = 500
              precision    recall  f1-score   support

         ham       0.95      0.99      0.97      1428
        spam       0.94      0.68      0.78       240

    accuracy                           0.95      1668
   macro avg       0.94      0.83      0.88      1668
weighted avg       0.95      0.95      0.94      1668

n-framm from 1 to 1
min_df = 0.0; max_df = 1.0; max_features = 500
              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1428
        spam       0.96      0.75      0.84       240

    accuracy                           0.96      1668
   macro avg       0.96      0.87      0.91      1668
weighted avg       0.96      0.96      0.96      1668

n-framm from 1 to 1
min_df = 0.0025; max_df = 0.01; max_features = 500
              precision    recall  f1-score   support

         ham       0.95      0.99      0.97      1428
        spam       0.94      0.

5.3) символьные n-граммы

In [ ]:
for i in range (3, 8):
  for j in range (i, 8):
    vectorizer = CountVectorizer(analyzer='char', ngram_range=(i, j))
    vectorized_x_train = vectorizer.fit_transform(x_train)
    clf = MultinomialNB()
    clf.fit(vectorized_x_train, y_train)

    vectorized_x_test = vectorizer.transform(x_test)
    pred = clf.predict(vectorized_x_test)
    print(f"n-gramm from {i} to {j}")
    print(classification_report(y_test, pred))

    history_element = []
    history_element.append("CountVectorizer - analyzer=char")
    history_element.append(1.0)
    history_element.append(1.0)
    history_element.append(None)
    history_element.append(f"({i}, {j})")
    weighted_avg = precision_recall_fscore_support(y_test, pred, average='weighted')
    history_element.append(weighted_avg[0])
    history_element.append(weighted_avg[1])
    history_element.append(weighted_avg[2])
    history_element.append(accuracy_score(y_test, pred))
    history.append(history_element)

n-gramm from 3 to 3
              precision    recall  f1-score   support

         ham       0.98      0.99      0.98      1428
        spam       0.91      0.86      0.88       240

    accuracy                           0.97      1668
   macro avg       0.94      0.92      0.93      1668
weighted avg       0.97      0.97      0.97      1668

n-gramm from 3 to 4
              precision    recall  f1-score   support

         ham       0.98      0.98      0.98      1428
        spam       0.91      0.89      0.90       240

    accuracy                           0.97      1668
   macro avg       0.94      0.94      0.94      1668
weighted avg       0.97      0.97      0.97      1668

n-gramm from 3 to 5
              precision    recall  f1-score   support

         ham       0.98      0.98      0.98      1428
        spam       0.90      0.89      0.90       240

    accuracy                           0.97      1668
   macro avg       0.94      0.94      0.94      1668
weighted avg  

Итог:

In [ ]:
df = pd.DataFrame(history, columns=['Vectorize', 'min_df', 'max_df', 'max_features', 'n-gramm', 'precision', 'recall', 'f1-score', 'accuracy'])
df.sort_values(by=["accuracy"], ascending=False).head(20)

,Vectorize,min_df,max_df,max_features,n-gramm,precision,recall,f1-score,accuracy
85,CountVectorizer - analyzer=char,1.0000,1.0,NaN,"(3, 6)",0.972251,0.972422,0.972325,0.972422
15,TfidfVectorizer,0.0000,1.0,1500.0,"(1, 1)",0.972527,0.971823,0.970584,0.971823
87,CountVectorizer - analyzer=char,1.0000,1.0,NaN,"(4, 4)",0.971873,0.971823,0.971847,0.971823
84,CountVectorizer - analyzer=char,1.0000,1.0,NaN,"(3, 5)",0.970482,0.970624,0.970547,0.970624
83,CountVectorizer - analyzer=char,1.0000,1.0,NaN,"(3, 4)",0.970399,0.970624,0.970495,0.970624
88,CountVectorizer - analyzer=char,1.0000,1.0,NaN,"(4, 5)",0.970251,0.970024,0.970127,0.970024
86,CountVectorizer - analyzer=char,1.0000,1.0,NaN,"(3, 7)",0.969835,0.970024,0.969919,0.970024
27,TfidfVectorizer,0.0000,1.0,1500.0,"(1, 2)",0.970829,0.970024,0.968606,0.970024
39,TfidfVectorizer,0.0000,1.0,1500.0,"(1, 3)",0.969920,0.968825,0.967210,0.968825
94,CountVectorizer - analyzer=char,1.0000,1.0,NaN,"(6, 6)",0.968814,0.968225,0.968468,0.968225


## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**Ответ** 

Как сказано вначале, метод возвращает список всех найденных *непересекающихся* совпадений. Так как метод нашёл 'abca', то 'abcx' он уже не может выделить - они имеют пересечение в тексте в виде подстроки 'a'.

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
result_findall = re.findall('(?:\s|^)\w{2}', 'Example: a Docker image is a recipe for running a containerized process.')
result_findall

['Ex', ' Do', ' im', ' is', ' re', ' fo', ' ru', ' co', ' pr']

### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
sentence_1 = 'Docker is a Linux container management toolkit with a “social” aspect, letting users publish container images and consume those published by others.'
sentence_2 = ' A Docker image is a recipe for running a containerized process.'
sentence_3 = ' In this guide, we build one for a simple Spring boot application.'
sentence_4 = ' If you are NOT using a Linux machine, you need a virtualized server.'
sentence_5 = ' If you install VirtualBox, other tools like the Mac’s boot2docker can seamlessly manage it for you.'
sentence_6 = ' Visit VirtualBox’s download site and pick the version for your machine.'
sentence_7 = ' Download and install.'
sentence_8 = ' Do not worry about actually running it.'
text = sentence_1 + sentence_2 + sentence_3 + sentence_4 + sentence_5 + sentence_6 + sentence_7 + sentence_8
result = re.split('\.', text, maxsplit=2)
print(result)

['Docker is a Linux container management toolkit with a “social” aspect, letting users publish container images and consume those published by others', ' A Docker image is a recipe for running a containerized process', ' In this guide, we build one for a simple Spring boot application. If you are NOT using a Linux machine, you need a virtualized server. If you install VirtualBox, other tools like the Mac’s boot2docker can seamlessly manage it for you. Visit VirtualBox’s download site and pick the version for your machine. Download and install. Do not worry about actually running it.']


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
result = re.sub('[0-9]', 'DIG', 'Код для входа в приложение: 1234.')
print(result)

Код для входа в приложение: DIGDIGDIGDIG.


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
result = re.sub('(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w\.-]*)*\/?', '', 'регулярные выражения онлайн: https://regex101.com/')
result

'регулярные выражения онлайн: '

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog = re.compile('[А-Яа-яё\-]{4,}')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'больше', 'больше', 'слов', 'Что-то']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
prog = re.compile('@[a-z0-9_\.-]+\.[a-z\.]{2,6}')
prog.findall("abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz")

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']